# Basics of pySAS

The version of pySAS available on GitHub is a development version. It will have features and capabilites not found in the standard distribution of pySAS that comes with SAS v. 21. The development cycle for the version on GitHub is by necessity much shorter then the development cycle for the full SAS.

First we will go over the basics of pySAS for the standard version and then the features that are common between the standard version and the development version. Then we will highlingting the additional features in the development version.

## Initializing pySAS

### Standard Version of pySAS
To use the standard version of pySAS the user must first initialize HEASOFT and SAS on the command line. The standard SAS environment variables must be set including `SAS_DIR`, `SAS_CCFPATH`, `SAS_PATH`, `SAS_VERBOSITY`, `SAS_SUPPRESS_WARNING`, and `SAS_IMAGEVIEWER`. Then the envionment variables `PATH`, `LIBRARY_PATH`, `LD_LIBRARY_PATH`, `PERL5LIB`, and `PYTHONPATH` must be ammended and then the user must set `SAS_CCF` and `SAS_ODF` for the desired observation. Most of this can be acomplished using a script such as `setsas.sh`. 

[More details can be found in the SAS Startup Thread in Python on the main XMM website](https://www.cosmos.esa.int/web/xmm-newton/sas-thread-startup-in-python).

After these enviornment variables are set, then a Python session can be started and pySAS imported using:
```python
import pysas
```

### Development Version of pySAS
Like the standard version, HEASOFT must first be initialized, but to begin using the development version the user just needs to start a Python session and type:

```python
import pysas
```
That's it! pySAS handles the rest! It will auto initialize SAS and set all environment variables for SAS!

## Calling SAS Tasks

The method for calling SAS tasks is the same between the standard version of pySAS and the development version. All standard SAS tasks are called using the wrapper function in pySAS.
```python
from pysas.wrapper import Wrapper as w
```
The wrapper takes two inputs, the first is the name of the SAS task as a string. The second is a list with the input arguments for the SAS task. Each input argument must be a string. If there are no input arguments then the list will be empty:
```python
w('sasver', []).run()
```

Each input argument must be a separate string in the list:
```python
inargs = ['table=event_list.fits', 
          'withfilteredset=yes', 
          "expression='(PATTERN <= 12)&&(PI in [200:4000])&&#XMMEA_EM'", 
          'filteredset=filtered_event_list.fits', 
          'filtertype=expression', 
          'keepfilteroutput=yes', 
          'updateexposure=yes', 
          'filterexposure=yes']

w('evselect', inargs).run()
```
<div class="alert alert-block alert-warning">
    <b>Note:</b> Some inputs require single quotes to be preserved in the string. This can be done using double quotes to form the string. i.e. "expression='(PATTERN <= 12)&&(PI in [200:4000])&&#XMMEA_EM'"
</div>

## Downloading Observation Data Files

### Standard Version of pySAS
Once SAS has been properly initalized ODF files for a specific Obs ID can be downloaded by running:
```python
work_dir = 'absolute_path_to_wrk_directory'
obsid = '##########'
inargs = [f'odfid={obsid}',f'workdir={work_dir}']
w('startsas', inargs).run()
```

The user must insert the desired obsID. This will download the ODF files for the Obs ID using the Python module astroquery. The task `startsas` will also run the tasks `cifbuild` and `odfingest`. 

[More details can be found in the SAS Startup Thread in Python on the main XMM website](https://www.cosmos.esa.int/web/xmm-newton/sas-thread-startup-in-python).

### Development Version of pySAS
To work with a particular observation the user then types:
```python
obsid = '##########'
odf = pysas.odfcontrol.ODFobject(obsid)
```
inserting the desired obsID. The development version of pySAS handles setting the envionment variables for the user. To download the data files the user then runs,
```python
odf.calibrate_odf()
```
This will download the files, then run the tasks `cifbuild` and `odfingest`. Alternatively the user could run,
```python
odf.basic_setup()
```

The function `basic_setup` will run `calibrate_odf` and then run `epproc`, `emproc`, and `rgsproc` with the option of also running `omichain`, and running `epchain` instead of `epproc` and `emchain` instead of `emproc`. 

[More information can be found in the documentation for odfcontol on GitHub](https://github.com/XMMGOF/pysas/blob/main/documentation/odfcontrol.ipynb).

If the ODF files have previously been downloaded and are in the default data directory, then upon creating the ODF object (i.e. "odf = pysas.odfcontrol.ODFobject(obsid)") pySAS will automatically find and link all important summary and calibration files, and also all previously generated event lists made by `epproc`, `emproc`, and `rgsproc`.

### Downloading Proprietary Data using the Standard Version of pySAS

**The standard version of pySAS cannot download proprietary data!!**

### Downloading Proprietary Data using the Development Version of pySAS

Proprietary data can be downloaded from both the XSA and the HEASARC. How each handles proprietary data is slightly different. For the XSA proprietary data requires your Cosmos username and password. To download proprietary data from the XSA using pySAS the user creates a ODF object like normal,

```python
obsid = '##########'
odf = pysas.odfcontrol.ODFobject(obsid)
```
Then when you run either `basic_setup` or `calibrate_odf` just indicate that the Obs ID is proprietary.

```python
odf.basic_setup(proprietary=True)
```

The data request is made using astroquery and astroquery will ask for the user's Cosmos username and password. The [ESA module for astroquery](https://github.com/astropy/astroquery/blob/main/astroquery/esa/xmm_newton/core.py) allows many more inputs for specifying exactly what XMM data to download. All of those inputs can be passed in as additional arguments to `basic_setup` or `calibrate_odf`. Those inputs will be passed on to astroquery.

Proprietary data from the HEASARC comes encrypted. Anyone can download it, but you need an encryption key to unpack the data. The encryption key is a alphanumeric string with 30 characters. You provide the encryption key to either `basic_setup` or `calibrate_odf` and pySAS will handle the rest.

```python
odf.basic_setup(repo='heasarc',encryption_key='XXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
```

## Features Only in the Development Version

### Default Configuration

The development version of pySAS comes with a setup script, `setup_pysas.py`. The purpose of this script is so the user can set the pySAS defaults 

    sas_dir (required)
    sas_ccfpath (required)
    data_dir (optional, but strongly recommended)
        
Once the defaults are set by the user, SAS will automatically be initialized when pySAS is imported (`import pysas`).

The user can also optionally (and is strongly recommended to) set a default data directory (`data_dir`) where observation data files (odf) will be downloaded. A separate subdirectory will be made for each observation ID (obsID) inside the data directory.
    
The default data directory can be set or changed later using the function `set_sas_config_option()`.

For example:
```python
from pysas.configutils import set_sas_config_option
data_path = '/path/to/new/data/dir/'
set_sas_config_option('data_dir', data_path)
```

The default values for the SAS directory (`sas_dir`), the path to the calibration files (`sas_ccfpath`), along with `verbosity` and `suppress_warning`, can also be set in the same way.

At any time the user can clear all previous values with,
```python
from pysas.configutils import clear_sas_defaults
clear_sas_defaults()
```

pySAS uses the standard Python configuration module [configparser](https://docs.python.org/3/library/configparser.html).

The configuration file for SAS can be found in the user's `HOME` directory (or the `XDG_CONFIG_HOME` directory which is usually the user's `HOME` directory) inside the hidden directory `.config`, then inside the `sas` directory. The configuration file will be named `sas.cfg`. It can be opened and edited by any text editor.

The typical contents of the file might be:
```
[DEFAULT]
sas_dir = /does/not/exist
sas_ccfpath = /does/not/exist
data_dir = /does/not/exist
verbosity = 4
suppress_warning = 1
on_sci_server = False

[sas]
data_dir = /home/rtanner/xmm_data
sas_dir = /home/rtanner/XMMGOF/xmmsas_20230412_1735
sas_ccfpath = /home/rtanner/sas/calibration
```
The values in `[sas]` will override the `[DEFAULT]` values. These values can be edited by hand, but it is *strongly* recommended to use the function included with pySAS to change the default values:
```python
from pysas.configutils import set_sas_config_option
set_sas_config_option('option_to_set','value_for_option')
```

---
### odfcontrol

`odfcontrol` is a new pySAS module that makes it easier for the user to interface with pySAS. It contains useful functions along with links to important directories and files.

When a user creates an `ODF` object using:
```python
obsid = '##########'
odf = pysas.odfcontrol.ODFobject(obsid)
```
The `ODF` object will have a few important links and information by default. These include:

    - odfid    : The ObsID number used to create the ODF object.
    - data_dir : Path to base data directory where all XMM data files 
                 will be downloaded. (/path/to/data_dir)
    - obs_dir  : Path to directory containing the data files 
                 (obs_dir --> /path/to/data_dir/obsid).
    - odf_dir  : Path to directory with the raw observation data files 
                 (odf_dir --> obs_dir/ODF --> data_dir/obsid/ODF).
    - work_dir : The working directory when SAS tasks will be run 
                 (work_dir --> obs_dir/work --> data_dir/obsid/work).
    - sas_ccf  : Link to the 'ccf.cif' file, if it exists. By default the 
                 'ccf.cif' file will be created in the work_dir.
    - sas_odf  : Link to the '*SUM.SAS' file, if it exists. By default the 
                 '*SUM.SAS' file will be created in the work_dir.
    - If they exist, links to the basic event lists will be stored in a 
      dictionary named 'files'.
      
      The 'files' dictionary will have the following keys:
      
            'sas_ccf'
            'sas_odf'
            'PNevt_list'
            'M1evt_list'
            'M2evt_list'
            'R1evt_list'
            'R2evt_list'
            'OMevt_list'

There is an entire document explaing the features that can be found in [the pySAS documentation on GitHub](./odfcontrol.ipynb).

---
### Input Arguments

As noted above, for all SAS tasks, input arguments should be in a list, with each input argument a separate string. 

For example,
```python
inargs = ['table=event_list.fits', 
          'withfilteredset=yes', 
          "expression='(PATTERN <= 12)&&(PI in [200:4000])&&#XMMEA_EM'", 
          'filteredset=filtered_event_list.fits', 
          'filtertype=expression', 
          'keepfilteroutput=yes', 
          'updateexposure=yes', 
          'filterexposure=yes']

w('evselect', inargs).run()
```

The development version of pySAS allows inputs to be passed in as a Python dictionary. For example, the same list of input arguments shown above could be written like this:

```python
inargs = {'table'            : 'event_list.fits', 
          'withfilteredset'  : 'yes', 
          'expression'       : "'(PATTERN <= 12)&&(PI in [200:4000])&&#XMMEA_EM'", 
          'filteredset'      : 'filtered_event_list.fits', 
          'filtertype'       : 'expression', 
          'keepfilteroutput' : 'yes', 
          'updateexposure'   : 'yes', 
          'filterexposure'   : 'yes'}

w('evselect', inargs).run()
```

or it could even be written like this:

```python
inargs = {}
inargs['table']            = 'event_list.fits'
inargs['withfilteredset']  = 'yes'
inargs['expression']       = "'(PATTERN <= 12)&&(PI in [200:4000])&&#XMMEA_EM'"
inargs['filteredset']      = 'filtered_event_list.fits'
inargs['filtertype']       = 'expression'
inargs['keepfilteroutput'] = 'yes'
inargs['updateexposure']   = 'yes'
inargs['filterexposure']   = 'yes'

w('evselect', inargs).run()
```

Then if a single input needs to be changed and the same SAS task run again the user could simply use:

```python
inargs['expression'] = "'(PATTERN <= 12)&&(PI in [4000:12000])&&#XMMEA_EM'"
w('evselect', inargs).run()
```

The reason why this capability was added was for experimental reasons and to allow greater flexibility in interacting with pySAS.

While the values for all input arguments should be in string format, the development version of pySAS will accept numbers, as shown below. These numbers will be converted into strings automatically.

```python
inargs = {}
inargs['spectrumset'] = 'R1_spectra.fits'
inargs['rmfset']      = 'rmf1_file.fits'
inargs['evlist']      = 'R1_event_list'
inargs['emin']        = 0.4
inargs['emax']        = 2.5
inargs['rows']        = 4000

w('rgsrmfgen', inargs).run()
```

The user can get all the default parameters for any SAS task by using:
```python
task_name = 'insert_task_name'
inargs = pysas.param.get_input_params(task_name)
```

This will return a dictionary with all possible input arguments for a given SAS task, along with their default values.

<div class="alert alert-block alert-warning">
    <b>Note:</b> At the moment it is <b>NOT</b> recommended to use the dict returned by get_input_params as the input for calling a SAS task (i.e. w('evselect', inargs).run()). Currently the way pySAS checks the validity of the inputs may cause unexpected fatal results when <b>all</b> default parameters are passed in. This is a rather complicated problem and we are working on it.
</div>